In [1]:
import arcpy
import requests
import zipfile
import io

In [2]:
#DATA DOWNLOAD
arcpy.env.workspace = r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project"

link = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regional_parcels/shp_plan_regional_parcels.zip"
output = requests.post(link)
content=output.content
zipp = zipfile.ZipFile(io.BytesIO(content))

zipp.extractall(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Parcels")


##Download Dakota separately
link2 = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_co_dakota/plan_parcels/shp_plan_parcels.zip"
output = requests.post(link2)
content=output.content
zipp = zipfile.ZipFile(io.BytesIO(content))

zipp.extractall(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Parcels")


##Download Ramsey separately
link3 = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_co_ramsey/plan_parcel_data/shp_plan_parcel_data.zip"
output = requests.post(link3)
content=output.content
zipp = zipfile.ZipFile(io.BytesIO(content))

zipp.extractall(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Parcels")

##Download Transit Stops
link2 = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_transit_stops/shp_trans_transit_stops.zip"
output = requests.post(link2)
content=output.content
zipp = zipfile.ZipFile(io.BytesIO(content))

zipp.extractall(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Transit Points")


##Download high frequency route
link3 = r"https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metc/trans_hi_frequency_transit_servc/shp_trans_hi_frequency_transit_servc.zip"
output=requests.post(link3)
content=output.content
zipp=zipfile.ZipFile(io.BytesIO(content))

zipp.extractall(r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project")

In [9]:
##DATA CLEANING

##Select All inactive stops
arcpy.management.SelectLayerByAttribute("TransitStops", "NEW_SELECTION", "active_sta = 'Archive'", None)

## DELETE all inactive stops
arcpy.management.DeleteRows("TransitStops")

## Delete all stop stops where boarding or alighting cannot occur
arcpy.management.SelectLayerByAttribute("TransitStops", "NEW_SELECTION", "board_flag = 0 And alight_fla = 0", None)
arcpy.management.DeleteRows("TransitStops")

##Add type field - bus or lightrail
arcpy.management.AddFields("TransitStops", "Bus SHORT # # # #")
arcpy.management.AddFields("TransitStops", "Lightrail SHORT # # # #")

##Select all light rail stops- each light rail stop was selected manually, then all FID's 
## were put into the line below.
arcpy.management.SelectLayerByAttribute("TransitStops", "NEW_SELECTION", "FID = 4059 Or FID = 4060 Or FID = 4324 Or FID = 4325 Or FID = 4413 Or FID = 4414 Or FID = 4569 Or FID = 4570 Or FID = 4811 Or FID = 4812 Or FID = 4988 Or FID = 4989 Or FID = 5587 Or FID = 5637 Or FID = 5843 Or FID = 5844 Or FID = 6711 Or FID = 6975 Or FID = 6976 Or FID = 7731 Or FID = 7732 Or FID = 7834 Or FID = 7835 Or FID = 8044 Or FID = 8045 Or FID = 8481 Or FID = 8482 Or FID = 8558 Or FID = 8559 Or FID = 8738 Or FID = 8739 Or FID = 8742 Or FID = 8743 Or FID = 8744 Or FID = 8745 Or FID = 8855 Or FID = 8856 Or FID = 8858 Or FID = 8859 Or FID = 9016 Or FID = 9017 Or FID = 9099 Or FID = 9100 Or FID = 9157 Or FID = 9158 Or FID = 9305 Or FID = 9306 Or FID = 9497 Or FID = 9498 Or FID = 9499 Or FID = 9500 Or FID = 41 Or FID = 42 Or FID = 545 Or FID = 546 Or FID = 766 Or FID = 767 Or FID = 1083 Or FID = 1084 Or FID = 1175 Or FID = 1176 Or FID = 2146 Or FID = 2147 Or FID = 2494 Or FID = 2495 Or FID = 2824 Or FID = 2825 Or FID = 2977 Or FID = 2978 Or FID = 3094 Or FID = 3095 Or FID = 3828 Or FID = 3829", None)

##Then give them a value of 1 for lightrail field
arcpy.management.CalculateField("TransitStops", "Lightrail", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

## Select all bus stops and give a value of 1
arcpy.management.SelectLayerByAttribute("TransitStops", "NEW_SELECTION", "Bus = 0 And Lightrail = 0", None)
arcpy.management.CalculateField("TransitStops", "Bus", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

## Spatial Join Transit Points with High Frequency Routes shapefile Within distance of 12 meters.
arcpy.analysis.SpatialJoin("TransitStops_Copy", "HighFrequencyNetwork", r"C:\Users\lbmaj\OneDrive\Documents\ArcGIS\Projects\Final Project\Final Project.gdb\TransitStops_SpatialJoin", "JOIN_ONE_TO_ONE", "KEEP_ALL", "WITHIN_A_DISTANCE", "12 Meters", '')

##Add new field HighFreq
arcpy.management.AddField("TransitStops_SpatialJoin", "HighFreq", "SHORT", None, None, None, '', "NULLABLE", "NON_REQUIRED", '')

## Set HighFreq to 1 if transit stop joined with High Frequency Network shape file
arcpy.management.SelectLayerByAttribute("TransitStops_SpatialJoin", "NEW_SELECTION", "Join_Count > 0", None)
arcpy.management.CalculateField("TransitStops_SpatialJoin", "HighFreq", "1", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

## Set HighFreq to 0 else
arcpy.management.SelectLayerByAttribute("TransitStops_SpatialJoin", "NEW_SELECTION", "Join_Count = 0", None)
arcpy.management.CalculateField("TransitStops_SpatialJoin", "HighFreq", "0", "PYTHON3", '', "TEXT", "NO_ENFORCE_DOMAINS")

## Remove join count field for clarity
arcpy.management.DeleteField("TransitStops_SpatialJoin", "Join_Count", "DELETE_FIELDS")

<Result 'TransitStops'>